<a href="https://colab.research.google.com/github/stakunlena/ich_final_project/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предобработка данных

## Загрузка библиотек и исходных данных

In [305]:
# Импортируем основные библиотеки
import pandas as pd
import numpy as np
from datetime import datetime

# Загружаем исходные данные
import os
from google.colab import drive # Импортируем библиотеку для работы с Google Drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Путь к папке с данными
base_path = '/content/drive/MyDrive/P. Project 07.11/Data/'

# Загружаем файлы в датафреймы
df_calls = pd.read_excel(base_path + 'Calls (Done).xlsx')
df_contacts = pd.read_excel(base_path + 'Contacts (Done).xlsx')
df_deals = pd.read_excel(base_path + 'Deals (Done).xlsx')
df_spend = pd.read_excel(base_path + 'Spend (Done).xlsx')

# Определим словарь: имя переменной → объект DataFrame
# которым будем пользоваться для групповых операций с датафреймами
dfs = {
    'df_contacts': df_contacts,
    'df_calls': df_calls,
    'df_deals': df_deals,
    'df_spend': df_spend
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Кастомные функции

In [306]:
import re

def clean_columns_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует названия столбцов DataFrame в формат snake_case.

    В текущих датасетах выполняются следующие преобразования:
    1. Удаление лишних пробелов по краям.
    2. Замена пробелов и круглых скобок на символ подчёркивания.
    3. Приведение всех символов к нижнему регистру.

    Параметры
    ----------
    df : pandas.DataFrame
        Исходный DataFrame с оригинальными названиями столбцов.

    Возвращает
    ----------
    pandas.DataFrame
        Тот же объект DataFrame с обновлёнными названиями столбцов.
    """
    df.columns = (
        df.columns
        .map(lambda x: str(x).strip()) # обрезаем пробелы и защищаем от NaN
        .map(lambda x: re.sub(r'[\s()]+', '_', x)) # пробелы и скобки → "_"
        .str.lower() # всё в нижний регистр
    )
    return df

def normalize_id_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Приводит все столбцы, содержащие идентификаторы (id), к строковому типу
    и очищает значения от артефактов Excel и текстовых маркеров пропусков.

    Функция автоматически находит все колонки, кроме перечисленных в списке
    исключений, в названии которых присутствует
    подстрока 'id' (без учёта регистра), и выполняет следующие преобразования:
      1. приведение к типу pandas.StringDtype() — NA-aware строковый тип;
      2. удаление пробелов по краям строк;
      3. удаление суффиксов '.0', появляющихся после экспорта в Excel;
      4. замена строковых маркеров пропусков ('nan', 'NaN', '') на pd.NA.

    Параметры
    ----------
    df : pd.DataFrame
        Входной датафрейм с данными, содержащими столбцы идентификаторов.

    Возвращает
    ----------
    pd.DataFrame
        Датафрейм с обновлёнными столбцами идентификаторов, приведёнными
        к строковому типу и очищенными от артефактов.

    Пример
    -------
    >>> df = pd.DataFrame({
    ...     "Id": [5805028000018777278.0, 5805028000018777000.0, None],
    ...     "ContactID": ["5.805028E+18", "5805028000018777000", ""]
    ... })
    >>> normalize_id_columns(df)
                id          contactid
    0  5805028000018777278  5.805028E+18
    1  5805028000018777000  5805028000018777000
    2

    Примечание
    ----------
    Функция не удаляет пропуски и не восстанавливает значения.
    Её цель — унификация типа данных и формата хранения идентификаторов
    во всех таблицах проекта перед их объединением.
    """
    # Список исключений
    skip_cols = {"initial_amount_paid"}

    # Находим все колонки, содержащие подстроку "id", кроме тех, что в списке исключений
    id_cols = [c for c in df.columns if "id" in c.lower() and c.lower() not in skip_cols]

    # Приводим найденные колонки к единому типу и формату
    for col in id_cols:
        df[col] = (
            df[col]
            .astype("string")                             # безопасный NA-aware тип
            .str.strip()                                  # удаляем пробелы по краям
            .str.replace(r"\.0$", "", regex=True)          # убираем .0 после Excel
            .replace({"nan": pd.NA, "NaN": pd.NA, "": pd.NA})  # нормализуем пропуски
        )

    return df

def analyze_missing_ratio(df: pd.DataFrame, column: str, threshold: float = 0.01) -> None:
    """
    Анализирует долю пропусков в заданной колонке датафрейма и выводит
    рекомендации по дальнейшей обработке строк с пропущенными значениями.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм для анализа.
    column : str
        Название столбца, в котором нужно проверить пропуски.
    threshold : float, optional
        Пороговое значение доли пропусков (по умолчанию 0.1 = 10%).
        Если доля пропусков меньше порога — рекомендуется удалить строки.
        Если доля выше порога — рекомендуется сохранить строки.

    Возвращает
    ----------
    None
        Функция только выводит информацию и не изменяет датафрейм.

    Пример
    -------
    >>> analyze_missing_ratio(df_calls, "contactid", threshold=0.1)
    Количество строк с пропусками в колонке 'contactid': 3933
    Доля пропусков от общего числа строк: 4.10%
    Рекомендация: доля пропусков меньше порогового значения (10.00%) → строки можно удалить.
    """
    # Проверка, что колонка существует
    if column not in df.columns:
        print(f"Ошибка: в датафрейме нет колонки '{column}'.")
        return

    # Подсчёт пропусков
    missing_rows = df[df[column].isna()]
    total_rows = len(df)
    missing_count = len(missing_rows)
    missing_share = missing_count / total_rows if total_rows > 0 else 0

    # Вывод статистики
    print(f"Количество строк с пропусками в колонке '{column}': {missing_count}")
    print(f"Доля пропусков от общего числа строк: {missing_share:.2%}")

    # Формирование рекомендации
    if missing_share < threshold:
        print(f"Рекомендация: доля пропусков меньше порогового значения ({threshold:.2%}) → строки можно удалить.")
    else:
        print(f"Рекомендация: доля пропусков превышает пороговое значение ({threshold:.2%}) → строки лучше сохранить.")

def drop_full_duplicates(df: pd.DataFrame, df_name: str = "DataFrame") -> pd.DataFrame:
    """
    Проверяет и удаляет полные дубликаты строк в переданном датафрейме.

    Функция подсчитывает количество полностью совпадающих строк,
    выводит информацию о результатах проверки и возвращает очищенный датафрейм.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм, в котором нужно найти и удалить дубликаты.
    df_name : str, optional
        Название датафрейма (для удобного вывода в логах), по умолчанию "DataFrame".

    Возвращает
    ----------
    pd.DataFrame
        Копия исходного датафрейма без полных дублей, с обновлёнными индексами.
    """
    duplicates_count = df.duplicated().sum()
    print(f"[{df_name}] Найдено полных дублей: {duplicates_count}")

    if duplicates_count > 0:
        df = df.drop_duplicates().reset_index(drop=True)
        print(f"[{df_name}] Полные дубли удалены. Размер после очистки: {df.shape}")
    else:
        print(f"[{df_name}] Полных дублей не обнаружено. Размер датафрейма: {df.shape}")

    return df

def convert_datetime_columns(df: pd.DataFrame, date_columns: list[str], df_name: str = "DataFrame") -> pd.DataFrame:
    """
    Преобразует указанные столбцы датафрейма в формат datetime64[ns].

    Функция безопасно приводит текстовые значения к типу datetime,
    используя формат '%d.%m.%Y %H:%M' (CRM-формат),
    и сообщает количество некорректных значений.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм для преобразования.
    date_columns : list[str]
        Список названий столбцов, которые нужно преобразовать.
    df_name : str, optional
        Название датафрейма (для логирования), по умолчанию "DataFrame".

    Возвращает
    ----------
    pd.DataFrame
        Копия датафрейма с обновлёнными типами указанных столбцов.
    """
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format="%d.%m.%Y %H:%M", errors="coerce")
            invalid = df[col].isna().sum()
            print(f"[{df_name}] Столбец '{col}' преобразован в datetime64[ns]. Некорректных значений: {invalid}")
        else:
            print(f"[{df_name}] Столбец '{col}' не найден, пропускаем.")
    return df

def export_dataframe_to_csv(df: pd.DataFrame, df_name: str, folder_path: str = "/content/drive/MyDrive/P. Project 07.11/csv/") -> str:
    """
    Экспортирует очищенный датафрейм в CSV-файл с меткой времени.

    Функция сохраняет переданный датафрейм в указанный каталог.
    Если папка не существует, она создаётся автоматически.
    Имя файла формируется в формате:
        <df_name>_clean_<YYYYMMDD_HHMM>.csv

    Параметры
    ----------
    df : pd.DataFrame
        Датафрейм, который нужно сохранить.
    df_name : str
        Имя датафрейма (используется в названии файла).
    folder_path : str, optional
        Путь к папке для сохранения. По умолчанию: "/content/drive/MyDrive/P. Project 07.11/csv/"

    Возвращает
    ----------
    str
        Полный путь к сохранённому файлу.
    """

    from datetime import datetime
    import os

    # Создаём папку, если её нет
    os.makedirs(folder_path, exist_ok=True)

    # Формируем имя файла с меткой времени
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    output_path = os.path.join(folder_path, f"{df_name}_clean_{timestamp}.csv")

    # Сохраняем CSV
    df.to_csv(output_path, index=False, sep=";", encoding="utf-8-sig")

    print(f"Файл '{df_name}' успешно сохранён по пути:\n{output_path}")
    return output_path


## Ревью исходных данных. Определение количества пропусков

In [307]:
# Вывод информации по каждому датафрейму в цикле по элементам словаря dfs
for name, df in dfs.items():
    print(f"\n{name}")
    #print(f"  Размер: {df.shape[0]} строк × {df.shape[1]} столбцов")
    #print(f"  Колонки: {df.columns.tolist()}")
    print(f"\n")
    print(f"  Информация о датасете:")
    print(df.info())
    print(f"\n")
    print(f"  Первые 5 строк датасета:")
    display(df.head())
    print(f"\n")
    print(f"  Количество пропусков в данных:")
    print(df.isna().sum())
    print(f"\n================================================")


df_contacts


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  18548 non-null  int64 
 1   Contact Owner Name  18548 non-null  object
 2   Created Time        18548 non-null  object
 3   Modified Time       18548 non-null  object
dtypes: int64(1), object(3)
memory usage: 579.8+ KB
None


  Первые 5 строк датасета:


,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,27.06.2023 11:28,22.12.2023 13:34
1,5805028000000872003,Charlie Davis,03.07.2023 11:31,21.05.2024 10:23
2,5805028000000889001,Bob Brown,02.07.2023 22:37,21.12.2023 13:17
3,5805028000000907006,Bob Brown,03.07.2023 05:44,29.12.2023 15:20
4,5805028000000939010,Nina Scott,04.07.2023 10:11,16.04.2024 16:14




  Количество пропусков в данных:
Id                    0
Contact Owner Name    0
Created Time          0
Modified Time         0
dtype: int64


df_calls


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  int64  
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  float64
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95791 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Dialled Number              0 non-null      float64
 8   Outgoing Call Status        86875 non-null  object 
 9   Scheduled in CRM            86875 non-null  float64
 10  Tag                 

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Dialled Number,Outgoing Call Status,Scheduled in CRM,Tag
0,5805028000000805001,30.06.2023 08:43,John Doe,NaN,Inbound,171.0,Received,NaN,NaN,NaN,NaN
1,5805028000000768006,30.06.2023 08:46,John Doe,NaN,Outbound,28.0,Attended Dialled,NaN,Completed,0.0,NaN
2,5805028000000764027,30.06.2023 08:59,John Doe,NaN,Outbound,24.0,Attended Dialled,NaN,Completed,0.0,NaN
3,5805028000000787003,30.06.2023 09:20,John Doe,5.805028e+18,Outbound,6.0,Attended Dialled,NaN,Completed,0.0,NaN
4,5805028000000768019,30.06.2023 09:30,John Doe,5.805028e+18,Outbound,11.0,Attended Dialled,NaN,Completed,0.0,NaN




  Количество пропусков в данных:
Id                                0
Call Start Time                   0
Call Owner Name                   0
CONTACTID                      3933
Call Type                         0
Call Duration (in seconds)       83
Call Status                       0
Dialled Number                95874
Outgoing Call Status           8999
Scheduled in CRM               8999
Tag                           95874
dtype: int64


df_deals


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21595 entries, 0 to 21594
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   21593 non-null  float64
 1   Deal Owner Name      21564 non-null  object 
 2   Closing Date         14645 non-null  object 
 3   Quality              19340 non-null  object 
 4   Stage                21593 non-null  object 
 5   Lost Reason          16124 non-null  object 
 6   Page

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,...,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch
0,5.805028e+18,Ben Hall,NaN,NaN,New Lead,NaN,/eng/test,03.07.23women,NaN,v16,...,NaN,NaN,21.06.2024 15:30,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN
1,5.805028e+18,Ulysses Adams,NaN,NaN,New Lead,NaN,/at-eng,NaN,NaN,NaN,...,Web Developer,Morning,21.06.2024 15:23,6.0,NaN,0,2000,5.805028e+18,NaN,NaN
2,5.805028e+18,Ulysses Adams,21.06.2024,D - Non Target,Lost,Non target,/at-eng,engwien_AT,00:26:43,b1-at,...,NaN,NaN,21.06.2024 14:45,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN
3,5.805028e+18,Eva Kent,21.06.2024,E - Non Qualified,Lost,Invalid number,/eng,04.07.23recentlymoved_DE,01:00:04,bloggersvideo14com,...,NaN,NaN,21.06.2024 13:32,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN
4,5.805028e+18,Ben Hall,21.06.2024,D - Non Target,Lost,Non target,/eng,discovery_DE,00:53:12,website,...,NaN,NaN,21.06.2024 13:21,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN




  Количество пропусков в данных:
Id                         2
Deal Owner Name           31
Closing Date            6950
Quality                 2255
Stage                      2
Lost Reason             5471
Page                       2
Campaign                5528
SLA                     6062
Content                 7448
Term                    9141
Source                     2
Payment Type           21099
Product                18003
Education Type         18295
Created Time               2
Course duration        18008
Months of study        20755
Initial Amount Paid    17430
Offer Total Amount     17410
Contact Name              63
City                   19084
Level of Deutsch       20344
dtype: int64


df_spend


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20779 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         20779 non-null  datetim

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
0,2023-07-03,Google Ads,gen_analyst_DE,6,0.00,0,NaN,NaN
1,2023-07-03,Google Ads,performancemax_eng_DE,4,0.01,1,NaN,NaN
2,2023-07-03,Facebook Ads,NaN,0,0.00,0,NaN,NaN
3,2023-07-03,Google Ads,NaN,0,0.00,0,NaN,NaN
4,2023-07-03,CRM,NaN,0,0.00,0,NaN,NaN




  Количество пропусков в данных:
Date              0
Source            0
Campaign       5994
Impressions       0
Spend             0
Clicks            0
AdGroup        6828
Ad             6828
dtype: int64



## Приведение названия колонок всех датасетов к snake_case

In [308]:
# Изменение названий колонок в датафреймах с помощью функции clean_columns_names
for name, df in dfs.items():
    clean_columns_names(df)
    print(f"\n{name}")
    print(list(df.columns))


df_contacts
['id', 'contact_owner_name', 'created_time', 'modified_time']

df_calls
['id', 'call_start_time', 'call_owner_name', 'contactid', 'call_type', 'call_duration_in_seconds_', 'call_status', 'dialled_number', 'outgoing_call_status', 'scheduled_in_crm', 'tag']

df_deals
['id', 'deal_owner_name', 'closing_date', 'quality', 'stage', 'lost_reason', 'page', 'campaign', 'sla', 'content', 'term', 'source', 'payment_type', 'product', 'education_type', 'created_time', 'course_duration', 'months_of_study', 'initial_amount_paid', 'offer_total_amount', 'contact_name', 'city', 'level_of_deutsch']

df_spend
['date', 'source', 'campaign', 'impressions', 'spend', 'clicks', 'adgroup', 'ad']


## Обработка PK и FK таблиц

Проанализируем первичные и вторичные ключи исходных таблиц:


| Датафрейм | Ключ (PK/FK) | Тип PK/FK | Количество<br>пропусков | Планируемые действия |
|-|-|-|-|-|
| df_calls | id (PK)| int64 | 0 | Преобразовать в string |
| df_calls | contactid (FK)| float64 | 3933 | Преобразовать в string с сохранением пропусков |
| df_contacts | id (PK) | int64 | 0 | Преобразовать в string |
| df_deals | id (PK) | float64 | 2 | Удалить строки с пропусками, затем преобразовать в string |
| df_spend | — | — | — | Отсутствует. Добавить? |

Выводы:
* в датафрейме df_calls обнаружены 3933 строк с пропусками во вторичном ключе contactid — их слишком много, чтобы удалить данные строки и надо преобразовать поле в формат string с сохранением пропусков;
* в датафрейме df_deals обнаружены 2 строки с пропусками в первичном ключе, скорее всего это «мусор» в данных, перед преобразованием в тип string их надо проанализировать и удалить, если они не содержат значимой информации;
* в датафрейме df_spend специальных полей с идентификаторами нет, возможно понадобится впоследствии добавить в нее первичный ключ.

### Анализ и обработка пропусков в contactid (FK) датафрейма df_calls

In [309]:
analyze_missing_ratio(df_calls, "contactid")

Количество строк с пропусками в колонке 'contactid': 3933
Доля пропусков от общего числа строк: 4.10%
Рекомендация: доля пропусков превышает пороговое значение (1.00%) → строки лучше сохранить.


Выводы:
* доля строк с пропущенным contactid слишком велика, чтобы удалять их исходных данных;
* следует преобразовать это поле к типу strig с сохранением пропусков.

### Анализ и обработка пропусков в PK датафрейма df_deals

In [310]:
# Отфильтровать строки с пропусками в Id
missing_deals_id_rows = df_deals[df_deals['id'].isna()]

# Посмотреть количество, их долю и сами строки
analyze_missing_ratio(df_calls, "id")
#print(f"Количество строк в df_deals с пропущенным Id: {len(missing_deals_id_rows)}\n")
#print(f"Доля строк с пропущенным Id: {len(missing_deals_id_rows) / len(df_deals):.2%}\n")
print(f"Содержимое строк с пропущенным Id:\n\n{missing_deals_id_rows}")

Количество строк с пропусками в колонке 'id': 0
Доля пропусков от общего числа строк: 0.00%
Рекомендация: доля пропусков меньше порогового значения (1.00%) → строки можно удалить.
Содержимое строк с пропущенным Id:

       id deal_owner_name closing_date quality stage lost_reason page  \
21593 NaN             NaN          NaN     NaN   NaN         NaN  NaN   
21594 NaN             NaN          NaN     NaN   NaN         NaN  NaN   

      campaign  sla content  ... product education_type created_time  \
21593      NaN  NaN     NaN  ...     NaN            NaN          NaN   
21594      NaN  NaN     NaN  ...     NaN          #REF!          NaN   

      course_duration months_of_study initial_amount_paid  offer_total_amount  \
21593             NaN             NaN                 NaN                 NaN   
21594             NaN             NaN                 NaN                 NaN   

       contact_name city level_of_deutsch  
21593           NaN  NaN              NaN  
21594          

Выводы по результатам анализа пропусков в PK в df_deals:
* в таблице df_deals есть 2 строки, в которых одновременно пропущены все ключевые поля: Id, Stage, Page, Source, Created Time (и фактически не содержат ни одного полезного значени);
* индексы этих строк: 21593 и 21594;
* это типичные артефакты Excel-выгрузки — «пустые строки внизу файла»;
* эти строки следует удалить как технический мусор;
* строка 21594 содержит в колонке Education Type содержится значение #REF! — это помешает удалить её методом dropna и перед этим надо заменить текстовые «артефакты Excel» (#REF!, #N/A, #VALUE!, nan, и т.д.) на настоящие пропуски (pd.NA)

In [311]:
# Заменяем текстовые артефакты Excel на NaN
df_deals = df_deals.replace(['#REF!', '#N/A', '#VALUE!', 'NaN', 'nan', ''], np.nan)

# Удаляем строки с пропусками в PK из df_deals
df_deals = df_deals.dropna(how='all').reset_index(drop=True)

# Проверяем результат
print(df_deals.isna().sum())

id                         0
deal_owner_name           29
closing_date            6948
quality                 2253
stage                      0
lost_reason             5469
page                       0
campaign                5526
sla                     6060
content                 7446
term                    9139
source                     0
payment_type           21097
product                18001
education_type         18294
created_time               0
course_duration        18006
months_of_study        20753
initial_amount_paid    17428
offer_total_amount     17408
contact_name              61
city                   19082
level_of_deutsch       20342
dtype: int64


### Преобразование первичных ключей таблиц к типу string

In [312]:
# Преобразуем столбцы-ключи в таблицах к типу string
for name, df in dfs.items():
    df = normalize_id_columns(df)
    id_cols = [c for c in df.columns if "id" in c.lower()]
    print(name.lower())
    print(f"{df[id_cols].dtypes}\n")

# Проверяем, что в преобразованном столбце сохранились строки с пропусками в contactid
analyze_missing_ratio(df_calls, "contactid")

df_contacts
id    string[python]
dtype: object

df_calls
id           string[python]
contactid    string[python]
dtype: object

df_deals
id                     string[python]
initial_amount_paid            object
dtype: object

df_spend
Series([], dtype: object)

Количество строк с пропусками в колонке 'contactid': 3933
Доля пропусков от общего числа строк: 4.10%
Рекомендация: доля пропусков превышает пороговое значение (1.00%) → строки лучше сохранить.


## Предобработка df_contacts (таблица Contacts (Done).xlsx)

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить датафрейм на наличие полностью идентичных строк с помощью кастомной функции drop_full_duplicates(). Удалить найденные дубликаты|Удалены повторяющиеся записи, каждая строка представляет уникальный контакт|
|2. Преобразование формата дат|Преобразовать столбцы created_time и modified_time к типу datetime64[ns] с указанием формата "%d.%m.%Y %H:%M"|Корректные типы данных, позволяющие выполнять сортировку и анализ по времени|
|3. Проверка логики дат|Проверить, что modified_time не раньше created_time, при нарушениях зафиксировать или удалить строки|Данные согласованы по времени, отсутствуют нелогичные даты|
|4. Очистка имён владельцев контактов и проверка на неявные дубли|Удалить лишние пробелы, заменить двойные пробелы одним, привести к единому регистру методом .str.title(). Сравнить уникальные значения contact_owner_name, при необходимости создать словарь нормализации|Единый формат имён владельцев, готовых для дальнейшего сопоставления с другими таблицами. Исправлены возможные опечатки и вариации написания, снижено количество уникальных значений|
|5. Финальная проверка типов данных|Убедиться, что id имеет строковый тип, contact_owner_name — строковый, даты — datetime|Все столбцы имеют корректные типы данных для анализа и объединений|
|6. Выгрузка очищенного датасета в .csv |Экспортировать датафрейм в файл .csv. Если в папке проекта нет подкаталога /csv то создать его|Файл сохранен в подкаталоге /csv в папке проекта|


In [313]:
print("=== 1. Проверка и удаление полных дублей ===\n")
df_contacts = drop_full_duplicates(df_contacts, "df_contacts")

print("\n=== 2. Преобразование формата дат ===\n")
df_contacts = convert_datetime_columns(
    df_contacts,
    ["created_time", "modified_time"],
    df_name="df_contacts"
)

print("\n=== 3. Проверка логики дат ===\n")
invalid_dates = df_contacts[df_contacts["modified_time"] < df_contacts["created_time"]]
invalid_count = len(invalid_dates)

if invalid_count == 0:
    print("Все строки корректны: modified_time не раньше created_time.")
else:
    print(f"Найдено строк с нарушением логики дат: {invalid_count}")
    display(invalid_dates)
# df_contacts = df_contacts[df_contacts["modified_time"] >= df_contacts["created_time"]].reset_index(drop=True)

print("\n=== 4. Очистка имён владельцев контактов и проверка на неявные дубли ===\n")
unique_count = df_contacts["contact_owner_name"].nunique()
print(f"Количество уникальных имён владельцев контактов: {unique_count}")
print(df_contacts["contact_owner_name"].unique())
print("\nОчистка имен владельцев контактов не требуется.")
print("Неявные дубли отсутствуют.")
print("Обнаружен 1 артефакт — значение False.")

count_false = (df_contacts["contact_owner_name"] == False).sum()
print(f"Количество строк с артефактом: {count_false}")
print("Доля пропусков меньше порогового значения (1%) → строки можно удалить.")

df_contacts = df_contacts[df_contacts["contact_owner_name"] != False].reset_index(drop=True)
# Преобразование имен к типу string
df_contacts["contact_owner_name"] = df_contacts["contact_owner_name"].astype("string")

print("\n=== 5. Финальная проверка типов данных ===\n")
print("Текущие типы данных столбцов:\n")
print(f"[df_contacts] Размер датафрейма: {df_contacts.shape}\n")
print(df_contacts.dtypes)

print("\n=== 6. Выгрузка очищенного датасета в .csv ===\n")
export_dataframe_to_csv(df_contacts, "df_contacts")

=== 1. Проверка и удаление полных дублей ===

[df_contacts] Найдено полных дублей: 0
[df_contacts] Полных дублей не обнаружено. Размер датафрейма: (18548, 4)

=== 2. Преобразование формата дат ===

[df_contacts] Столбец 'created_time' преобразован в datetime64[ns]. Некорректных значений: 0
[df_contacts] Столбец 'modified_time' преобразован в datetime64[ns]. Некорректных значений: 0

=== 3. Проверка логики дат ===

Все строки корректны: modified_time не раньше created_time.

=== 4. Очистка имён владельцев контактов и проверка на неявные дубли ===

Количество уникальных имён владельцев контактов: 28
['Rachel White' 'Charlie Davis' 'Bob Brown' 'Nina Scott' 'Alice Johnson'
 'Ian Miller' 'Jane Smith' 'Julia Nelson' 'George King' 'Quincy Vincent'
 'Diana Evans' 'Kevin Parker' 'Ulysses Adams' 'Victor Barnes'
 'Yara Edwards' 'Paula Underwood' 'Mason Roberts' 'Ben Hall' 'Amy Green'
 'Cara Iverson' 'Oliver Taylor' 'Eva Kent' False 'Zachary Foster'
 'Sam Young' 'Wendy Clark' 'Tina Zhang' 'Derek J

'/content/drive/MyDrive/P. Project 07.11/csv/df_contacts_clean_20251026_2136.csv'

## Предобработка df_calls (таблица Calls (Done).xlsx)

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить датафрейм на наличие полностью идентичных строк с помощью кастомной функции drop_full_duplicates(). Удалить найденные дубликаты|Удалены повторяющиеся строки, каждая запись соответствует уникальному звонку|
|2. Преобразование формата даты и времени|Преобразовать столбец call_start_time в формат datetime64[ns] с указанием формата "%d.%m.%Y %H:%M". Проверить корректность диапазона дат и отсутствие будущих значений|Корректный тип данных, позволяющий сортировать и анализировать звонки по времени|
|3. Очистка имён владельцев звонков|Удалить лишние пробелы, заменить двойные пробелы одним, привести значения к единому регистру методом .str.title(). Проверить список уникальных имён на наличие опечаток и дублей|Единый формат имён сотрудников, готовый для анализа активности и объединений с другими таблицами|
|4. Восстановление пропусков contactid|Попытаться восстановить пропуски contactid по таблице contacts при совпадении владельца звонка с владельцем контакта|Максимально заполненные идентификаторы контактов без потери точности значений|
|5. Проверка и очистка категориальных значений|Проверить уникальные значения столбцов call_type, call_status, outgoing_call_status на опечатки и различия в регистре. Привести к унифицированному набору категорий|Корректные и стандартизированные категориальные значения, готовые к агрегации|
|6. Обработка длительности звонков|Проверить столбец call_duration_in_seconds_ на наличие пропусков и нулевых значений. Заменить NaN на 0. Проверить согласованность длительности с call_status (attended не может иметь 0 секунд)|Отсутствуют пропуски, длительность соответствует логике состояний звонков|
|7. Проверка несогласованных статусов|Проверить случаи, где одновременно присутствуют inbound и outbound звонки для одного контакта и времени. Добавить флаг is_multi_type_call для таких строк|Зафиксированы и помечены технические дубликаты событий CRM|
|8. Удаление полностью пустых столбцов|Удалить столбцы dialled_number и tag, содержащие 100% пропусков|Исключены неинформативные поля, датафрейм содержит только полезные данные|
|9. Проверка типов данных|Проверить, что id и contactid имеют строковый тип, call_start_time — datetime, количественные поля — числовой тип|Все поля имеют корректные типы для анализа и объединений|
|10. Выгрузка очищенного датасета в .csv|Экспортировать df_calls в файл .csv. Если в папке проекта отсутствует подкаталог /csv, создать его|Файл сохранён в подкаталоге /csv в папке проекта|

In [314]:
print("=== 1. Проверка и удаление полных дублей ===\n")

# Применяем кастомную функцию для очистки дублей
df_calls = drop_full_duplicates(df_calls, "df_calls")

print("\n=== 2. Преобразование формата даты и времени ===\n")

# Преобразуем столбец call_start_time в формат datetime
df_calls = convert_datetime_columns(
    df_calls,
    date_columns=["call_start_time"],
    df_name="df_calls"
)

print("\n=== 3. Проверка уникальных имён владельцев звонков ===\n")

unique_count = df_calls["call_owner_name"].nunique()
print(f"Количество уникальных имён владельцев звонков: {unique_count}")
print(df_calls["call_owner_name"].unique())
print("\nОчистка имен владельцев контактов не требуется.")
print("Неявные дубли отсутствуют.")
print("Артефакты не обнаружены.")

print("\n=== 4. Восстановление пропусков contactid ===\n")

# Считаем количество пропусков до восстановления
missing_before = df_calls["contactid"].isna().sum()
print(f"Пропусков до восстановления: {missing_before}")

# Создаём словарь соответствий: владелец контакта → id
owner_to_contactid = (
    df_contacts[["contact_owner_name", "id"]]
    .drop_duplicates(subset=["contact_owner_name"])
    .set_index("contact_owner_name")["id"]
    .to_dict()
)

# Маска строк с пропущенными contactid
mask_missing = df_calls["contactid"].isna()

# Заполняем пропуски contactid по совпадению владельцев
df_calls.loc[mask_missing, "contactid"] = (
    df_calls.loc[mask_missing, "call_owner_name"].map(owner_to_contactid)
)

# Проверяем результат
missing_after = df_calls["contactid"].isna().sum()
filled = missing_before - missing_after

print(f"Восстановлено contactid: {filled}")
print(f"Осталось пропусков: {missing_after}")

print("\n=== Контроль не найденных соответствий contactid ===\n")

# Находим строки, где contactid всё ещё отсутствует
missing_contacts = df_calls[df_calls["contactid"].isna()]

# Считаем количество владельцев звонков без contactid
owners_without_contactid = missing_contacts["call_owner_name"].unique()
count_owners = len(owners_without_contactid)
count_calls = len(missing_contacts)

print(f"Всего звонков без contactid: {count_calls}")
print(f"Количество уникальных владельцев без contactid: {count_owners}\n")

# Выводим список имён владельцев, у которых не найден contactid
print("Владельцы звонков без соответствующего contactid:")
print(sorted(owners_without_contactid))

print("\n=== 5. Проверка и очистка категориальных значений ===")

# Список категориальных столбцов
cat_cols = ["call_type", "call_status", "outgoing_call_status", "scheduled_in_crm"]

for col in cat_cols:
    if col in df_calls.columns:
        print(f"\n--- {col} ---")
        print(f"Уникальных значений: {df_calls[col].nunique(dropna=True)}")
        print("Список уникальных значений:")
        print(sorted(df_calls[col].dropna().unique()))

# Очистка строковых категорий: удаляем пробелы и унифицируем регистр
cols_to_clean = ["call_type", "call_status", "outgoing_call_status"]
for col in cols_to_clean:
    if col in df_calls.columns:
        df_calls[col] = (
            df_calls[col]
            .astype("string")
            .str.strip()
            .str.lower()
            .str.replace(r"\s+", " ", regex=True)
        )

# Проверим результат очистки
print("\nПосле нормализации регистра и пробелов и преобразования scheduled_in_crm в логический формат:")
for col in cols_to_clean:
    if col in df_calls.columns:
        print(f"\n--- {col} ---")
        print(sorted(df_calls[col].dropna().unique()))

# Преобразование scheduled_in_crm в логический формат ---
df_calls["scheduled_in_crm"] = (
    df_calls["scheduled_in_crm"]
    .fillna(0)        # на случай пропусков
    .astype("int8")   # компактный логический тип
)
print("\nСтолбец 'scheduled_in_crm' преобразован в логический формат (0/1)")
print(df_calls["scheduled_in_crm"].value_counts(dropna=False))

print("\n=== 6. Обработка длительности звонков ===\n")

# 6.1 Диагностика пропусков и базовой статистики
col = "call_duration_in_seconds_"

total_rows = len(df_calls)
missing = df_calls[col].isna().sum()
zero_dur = (df_calls[col] == 0).sum()

print(f"Всего строк: {total_rows}")
print(f"Пропусков в '{col}': {missing} ({missing / total_rows:.2%})")
print(f"Нулевая длительность: {zero_dur} ({zero_dur / total_rows:.2%})")

print("\nОсновные статистики по длительности (сек):")
print(df_calls[col].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99]).round(2))


# 6.2 Замена пропусков
# Пропуски заменяем на 0, так как они означают, что звонок не состоялся
df_calls[col] = df_calls[col].fillna(0)

print("\nПропуски заменены на 0. Проверим:")
print(df_calls[col].isna().sum(), "пропусков осталось")


# 6.3 Проверка логических аномалий
# Аномалия 1: статус completed, но длительность = 0
mask_completed_zero = (df_calls["call_status"] == "attended dialled") & (df_calls[col] == 0)
n_completed_zero = mask_completed_zero.sum()

# Аномалия 2: статус unattended, но длительность > 0
mask_unattended_positive = (df_calls["call_status"] == "unattended dialled") & (df_calls[col] > 0)
n_unattended_positive = mask_unattended_positive.sum()

print("\nПроверка логических несоответствий:")
print(f"Звонков со статусом 'attended dialled' и нулевой длительностью: {n_completed_zero}")
print(f"Звонков со статусом 'unattended dialled' и положительной длительностью: {n_unattended_positive}")


# 6.4 Проверка аномально длинных звонков
threshold = df_calls[col].quantile(0.99)
outliers = df_calls[df_calls[col] > threshold]

print(f"\nАномально длинные звонки (> 99 перцентиля = {threshold:.0f} сек): {len(outliers)}")
print(outliers[[col, "call_type", "call_status"]].head(10)) # подумать про выгрузку в отдельный файл

print("=== 7. Проверка несогласованных статусов (разные call_type при одинаковом времени и контакте) ===\n")

print("=== 8. Удаление полностью пустых столбцов (dialled_number и tag) ===\n")

# Удаляем неинформативные столбцы
df_calls = df_calls.drop(columns=["dialled_number", "tag"], errors="ignore")

print("Столбцы 'dialled_number' и 'tag' удалены. Текущий список колонок:")
print(df_calls.columns.tolist())

print("\n=== 9. Финальная проверка типов данных ===\n")

print("Текущие типы данных столбцов:\n")
print(f"[df_calls] Размер датафрейма: {df_calls.shape}\n")
print(df_calls.dtypes)

print("\n=== 10. Выгрузка очищенного датасета в .csv ===\n")
export_dataframe_to_csv(df_calls, "df_calls")

=== 1. Проверка и удаление полных дублей ===

[df_calls] Найдено полных дублей: 0
[df_calls] Полных дублей не обнаружено. Размер датафрейма: (95874, 11)

=== 2. Преобразование формата даты и времени ===

[df_calls] Столбец 'call_start_time' преобразован в datetime64[ns]. Некорректных значений: 0

=== 3. Проверка уникальных имён владельцев звонков ===

Количество уникальных имён владельцев звонков: 33
['John Doe' 'Jane Smith' 'Alice Johnson' 'Bob Brown' 'Charlie Davis'
 'Diana Evans' 'Ethan Harris' 'Fiona Jackson' 'George King' 'Hannah Lee'
 'Ian Miller' 'Julia Nelson' 'Kevin Parker' 'Laura Quinn' 'Mason Roberts'
 'Nina Scott' 'Oliver Taylor' 'Paula Underwood' 'Quincy Vincent'
 'Rachel White' 'Sam Young' 'Tina Zhang' 'Ulysses Adams' 'Victor Barnes'
 'Wendy Clark' 'Xander Dean' 'Yara Edwards' 'Zachary Foster' 'Amy Green'
 'Ben Hall' 'Cara Iverson' 'Derek James' 'Eva Kent']

Очистка имен владельцев контактов не требуется.
Неявные дубли отсутствуют.
Артефакты не обнаружены.

=== 4. Восстан

'/content/drive/MyDrive/P. Project 07.11/csv/df_calls_clean_20251026_2136.csv'

### Проверка логики показателя scheduled_in_crm

В большинстве CRM-систем:
* 1 — звонок был запланирован в календаре или задаче CRM (назначен заранее менеджером);
* 0 — звонок не был запланирован, а создан постфактум (например, ручной звонок или входящий).

Тем не менее, чтобы делать корректные выводы о статистике запланированных звонков требуется убедиться, что в нашем случае эта логика работает.

Для этого построим кросстабы по типам звонков и scheduled_in_crm и по статусам звонков и scheduled_in_crm.

In [315]:
print("=== Проверка логики scheduled_in_crm ===\n")

# Пересечение по типам звонков
print("=== Пересечение по типам звонков ===")
print(pd.crosstab(df_calls["call_type"], df_calls["scheduled_in_crm"]))

# Пересечение по статусам
print("\n=== Пересечение по статусам ===")
print(pd.crosstab(df_calls["call_status"], df_calls["scheduled_in_crm"]))


=== Проверка логики scheduled_in_crm ===

=== Пересечение по типам звонков ===
scheduled_in_crm      0    1
call_type                   
inbound            3078    0
missed             5921    0
outbound          86733  142

=== Пересечение по статусам ===
scheduled_in_crm                0   1
call_status                          
attended dialled            70703   0
cancelled                       0  20
missed                       5922   0
overdue                         0  60
received                     3077   0
scheduled                       0   3
scheduled attended              0  14
scheduled attended delay        0  22
scheduled unattended            0   6
scheduled unattended delay      0  17
unattended dialled          16030   0


Видим, что scheduled_in_crm = 1:
*   встречаются только у исходящих (outbound) звонков;
*   только у звонков со статусами отменено (cancelled), просроченный (overdue) и разнообразные виды запланировано (scheduled).

Вывод: логика значений scheduled_in_crm:
*   0 = «Нет»;
*   1 = «Да».



## Предобработка df_deals (таблица Deals (Done).xlsx)

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить датафрейм на наличие полностью идентичных строк с помощью df_deals.drop_duplicates(). Удалить найденные дубликаты|Удалены повторяющиеся записи, каждая строка представляет уникальную сделку|
|2. Преобразование формата дат|Преобразовать столбцы created_time и closing_date к типу datetime64[ns] с указанием формата "%d.%m.%Y %H:%M" для created_time и "%d.%m.%Y" для closing_date|Корректные типы данных, обеспечивающие возможность анализа по времени|
|3. Проверка логики дат|Проверить, что closing_date не раньше created_time. При нарушениях зафиксировать или удалить строки|Данные согласованы по времени, отсутствуют нелогичные даты|
|4. Очистка имён владельцев сделок|Удалить лишние пробелы, заменить двойные пробелы одним, привести имена к единому регистру методом .str.title()|Единый формат имён менеджеров, готовый для сопоставления с другими таблицами|
|5. Проверка орфографических вариантов имён|Проверить уникальные значения deal_owner_name и при необходимости создать словарь нормализации для исправления опечаток и сокращений|Исправлены возможные опечатки и вариации написания, уменьшено количество уникальных имён|
|6. Преобразование числовых столбцов|Преобразовать столбцы initial_amount_paid и offer_total_amount к типу float, а course_duration и months_of_study — к целому типу Int64|Корректные числовые типы, готовые к аналитическим вычислениям|
|7. Очистка категориальных полей|Удалить пробелы по краям и внутри строк, привести текстовые значения к единому регистру для полей stage, quality, payment_type, product, education_type, source, campaign, content, term, city, level_of_deutsch|Единообразие категориальных данных, снижение числа дубликатов по формату записи|
|8. Проверка поля stage на корректность|Проверить, что значения stage соответствуют этапам воронки продаж (Lead, Contacted, Demo, Payment Process, Active Student, Churned). Исправить ошибки или неточные значения|Корректная структура этапов сделки, единая логика для анализа конверсий|
|9. Проверка финансовых показателей|Проверить, что offer_total_amount больше либо равно initial_amount_paid. При нарушениях поменять значения в найденных строках местами|Финансовые данные согласованы, исключены некорректные сделки|
|10. Проверка поля quality|Проверить категорию качества сделки (например, Hot, Warm, Cold) на наличие опечаток и приведение к единому регистру|Корректное распределение сделок по качеству, без дублирующих вариантов записи|
|11. Проверка логики обучения|Проверить, что months_of_study не превышает course_duration. При нарушениях отметить строки для анализа|Логически согласованные данные по продолжительности обучения|
|12. Финальная проверка типов данных|Убедиться, что все поля имеют корректные типы: даты — datetime64[ns], суммы — float, категории — string|Данные готовы для объединения с другими таблицами и анализа|
|13. Выгрузка очищенного набора данных в CSV|Сохранить очищенный датафрейм в файл с именем df_deals_clean_YYYYMMDD_HHMM.csv в папку проекта csv/ с кодировкой utf-8-sig|Создан файл с очищенными данными, готовый к дальнейшему использованию и визуализации|


In [316]:
print("=== 1. Проверка и удаление полных дублей ===\n")
# Применяем кастомную функцию для очистки дублей
df_deals = drop_full_duplicates(df_deals, "df_deals")

print("\n=== 2. Преобразование формата дат ===\n")
# Преобразуем столбцы created_time и closing_date в формат datetime64[ns]
df_deals = convert_datetime_columns(
    df_deals,
    date_columns=["created_time", "closing_date"],
    df_name="df_deals"
)

print("\n=== 2.1 Коррекция столбца closing_date ===\n")

# Повторное преобразование closing_date без времени
df_deals["closing_date"] = pd.to_datetime(
    df_deals["closing_date"],
    format="%d.%m.%Y",
    errors="coerce"
)

# Проверяем количество пропусков
missing_closing = df_deals["closing_date"].isna().sum()
print(f"После преобразования 'closing_date' пропусков: {missing_closing}")

# Анализ стадий без даты закрытия
stage_missing = df_deals.loc[df_deals["closing_date"].isna(), "stage"].value_counts()
print("\nРаспределение стадий среди сделок без даты закрытия:\n")
print(stage_missing)

# Итоговый контроль
final_missing = df_deals["closing_date"].isna().sum()
print(f"\nИтоговое количество пропусков closing_date: {final_missing}")
print("\nТип данных closing_date:", df_deals["closing_date"].dtype)

print("\n=== 3. Проверка логики дат ===\n")

# Проверка логической согласованности дат:
# closing_date должна быть позже или равна created_time
invalid_dates = df_deals[df_deals["closing_date"] < df_deals["created_time"]]

# Подсчёт количества нарушений
invalid_count = len(invalid_dates)

# Вывод результата проверки
if invalid_count == 0:
    print("Все строки корректны: closing_date не раньше created_time.")
else:
    print(f"Найдено строк с нарушением логики дат: {invalid_count}")
    display(invalid_dates)

# (опционально) Удаление или корректировка некорректных строк
# df_deals = df_deals[df_deals["closing_date"] >= df_deals["created_time"]].reset_index(drop=True)


print("\n=== 4. Очистка имён владельцев сделок ===\n")

# Очистка и стандартизация имён менеджеров, ответственных за сделки
df_deals["deal_owner_name"] = (
    df_deals["deal_owner_name"]
    .astype("string")                # Приведение к строковому типу (NA-aware)
    .str.strip()                     # Удаление пробелов по краям
    .str.replace(r"\s+", " ", regex=True)  # Замена нескольких пробелов одним
    .str.title()                     # Приведение регистра к формату "Имя Фамилия"
)

# Проверка на наличие артефактов и дубликатов
unique_count = df_deals["deal_owner_name"].nunique()
print(f"Количество уникальных имён владельцев сделок: {unique_count}\n")

# Просмотр первых 20 уникальных значений
print("Примеры очищенных имён владельцев:")
print(df_deals["deal_owner_name"].dropna().unique()[:20])

# Проверка на возможные артефакты (например, False, NaN, пустые строки)
artifact_mask = df_deals["deal_owner_name"].isin([False, "False", "Nan", "None", "<NA>"])
artifact_count = artifact_mask.sum()

if artifact_count > 0:
    print(f"\nОбнаружено артефактных значений: {artifact_count}. Заменяем их на pd.NA.")
    df_deals.loc[artifact_mask, "deal_owner_name"] = pd.NA
else:
    print("\nАртефактные значения не обнаружены.")

# Контроль типов после очистки
print("\nТип данных столбца 'deal_owner_name':", df_deals["deal_owner_name"].dtype)

print("\n=== 5. Проверка орфографических вариантов имён ===\n")
print("Проверка орфографических и форматных вариантов не требуется.")

print("\n=== 6. Преобразование числовых полей ===\n")

# Определяем числовые поля
numeric_cols_float = ["initial_amount_paid", "offer_total_amount"]
numeric_cols_int = ["course_duration", "months_of_study"]

# Преобразуем денежные значения к float
for col in numeric_cols_float:
    if col in df_deals.columns:
        df_deals[col] = (
            df_deals[col]
            .astype(str)
            .str.strip()
            # удаляем разделители тысяч (точки между цифрами)
            .str.replace(r"(?<=\d)\.(?=\d{3}(\D|$))", "", regex=True)
            # заменяем запятую на точку (десятичный разделитель)
            .str.replace(",", ".", regex=False)
            # убираем все нечисловые символы, кроме точки
            .str.replace(r"[^\d.]", "", regex=True)
            # заменяем пустые строки и текстовые NaN на np.nan
            .replace(["", "nan", "NaN", "<NA>"], np.nan)
        )
        df_deals[col] = pd.to_numeric(df_deals[col], errors="coerce")
        print(f"Столбец '{col}' успешно преобразован к типу float.")

# Преобразуем целочисленные поля
for col in numeric_cols_int:
    if col in df_deals.columns:
        df_deals[col] = pd.to_numeric(df_deals[col], errors="coerce").astype("Int64")
        print(f"Столбец '{col}' преобразован к типу Int64 (nullable).")

# Проверим итоговую статистику
print("\n=== Проверка числовых полей после преобразования ===\n")
for col in numeric_cols_float + numeric_cols_int:
    if col in df_deals.columns:
        missing = df_deals[col].isna().sum()
        min_val = df_deals[col].min()
        max_val = df_deals[col].max()
        print(f"{col:22} | Пропусков: {missing:5} | Мин: {min_val} | Макс: {max_val}")

print("\n=== 6.1 Логическая проверка числовых полей ===\n")

# Проверка длительности курса
invalid_duration = df_deals[
    (df_deals["course_duration"] <= 0) | (df_deals["course_duration"] > 60)
]
print(f"Некорректная длительность курса: {len(invalid_duration)} записей")

# Проверка месяцев обучения
invalid_study = df_deals[
    (df_deals["months_of_study"] < 0) |
    (df_deals["months_of_study"] > df_deals["course_duration"])
]
print(f"Некорректное количество месяцев обучения: {len(invalid_study)} записей")

# Проверка сумм оплаты
invalid_payment = df_deals[
    (df_deals["initial_amount_paid"] > df_deals["offer_total_amount"]) |
    (df_deals["initial_amount_paid"] < 0)
]
print(f"Некорректные значения оплаты: {len(invalid_payment)} записей")

# Проверка отрицательных общих сумм
invalid_offer = df_deals[df_deals["offer_total_amount"] < 0]
print(f"Отрицательные значения общей суммы предложения: {len(invalid_offer)} записей")

# Вывод примеров ошибок
if len(invalid_duration) or len(invalid_study) or len(invalid_payment) or len(invalid_offer):
    print("\nПримеры некорректных записей (первые 10):")
    display(
        pd.concat([invalid_duration, invalid_study, invalid_payment, invalid_offer])
        .drop_duplicates()
        .head(10)
    )
else:
    print("Все числовые поля логически корректны.")


print("\n=== 7. Очистка категориальных полей ===\n")

# Цель этапа: очистить и унифицировать значения в категориальных полях CRM.
# Удаляются лишние пробелы, дублирующие пробелы, символы и различия в регистре.

# Перечень категориальных полей для очистки
cat_cols = [
    "stage", "quality", "payment_type", "product",
    "education_type", "source", "campaign", "content",
    "term", "city", "level_of_deutsch", "page", "lost_reason"
]

# Очистка и нормализация категориальных полей
for col in cat_cols:
    if col in df_deals.columns:
        df_deals[col] = (
            df_deals[col]
            .astype("string")
            .str.strip() # удаляем пробелы по краям
            .str.replace(r"\s+", " ", regex=True) # заменяем двойные пробелы одним
            .str.replace(r"[-_]", " ", regex=True) # нормализуем разделители
            .str.title() # приводим регистр к "Title Case"
        )
        print(f"Столбец '{col}' очищен и нормализован.")
    else:
        print(f"Столбец '{col}' отсутствует в датафрейме и пропущен.")

# Проверка количества уникальных значений в основных категориальных полях
print("\nКоличество уникальных значений после очистки:")
for col in ["stage", "quality", "payment_type", "product", "education_type"]:
    if col in df_deals.columns:
        print(f"{col:20}: {df_deals[col].nunique()} уникальных значений")

# Проверим наличие артефактов (False, NaN, <NA>)
print("\nПроверка на наличие артефактных значений:")
for col in cat_cols:
    if col in df_deals.columns:
        artifacts = df_deals[col].isin(["False", "None", "Nan", "<Na>", "Na"]).sum()
        if artifacts > 0:
            print(f"  {col:20}: обнаружено {artifacts} артефактных значений → заменяем на NA")
            df_deals.loc[df_deals[col].isin(["False", "None", "Nan", "<Na>", "Na"]), col] = pd.NA

# Контроль после очистки
print("\nТипы данных после очистки категориальных полей:\n")
print(df_deals[cat_cols].dtypes)

print("\n=== 8. Проверка и нормализация поля 'stage' (финальный словарь) ===\n")

# Цель этапа:
# Создать новый столбец stage_normalized с унифицированными стадиями воронки продаж,
# сохранив оригинальное значение stage для аудита.

# Список уникальных значений поля stage (для составления словаря маппинга)
unique_count = df_deals["stage"].nunique()
print(f"Количество уникальных значений поля stage: {unique_count}")
print(df_deals["stage"].unique())

#Эталонные стадии воронки онлайн-школы
valid_stages = [
    "Lead",
    "Contacted",
    "Demo",
    "Payment Process",
    "Active Student",
    "Churned"
]

# Финальный расширенный словарь нормализации
# категория — Need To Call Sales (с несколькими пробелами внутри строки)
# не совпала при первом подходе с ключом "Need To Call Sales"
#(с одним пробелом))

stage_mapping = {
    # Основные стандартные варианты
    "New Lead": "Lead",
    "Contact": "Contacted",
    "Demo Call": "Demo",
    "Trial Lesson": "Demo",
    "Payment": "Payment Process",
    "Payment In Progress": "Payment Process",
    "Paid": "Active Student",
    "Student": "Active Student",
    "Active": "Active Student",
    "Closed Lost": "Churned",
    "Lost": "Churned",
    "Inactive": "Churned",

    # Добавленные из выявленных нестандартных стадий
    "Call Delayed": "Lead",
    "Registered On Webinar": "Lead",
    "Registered On Offline Day": "Lead",
    "Need To Call": "Contacted",
    "Need To Call Sales": "Contacted",
    "Need To Call   Sales": "Contacted",  # исправленный вариант с лишними пробелами
    "Need A Consultation": "Demo",
    "Test Sent": "Demo",
    "Qualificated": "Contacted",
    "Waiting For Payment": "Payment Process",
    "Payment Done": "Active Student",
    "Free Education": "Active Student"
}

# Создаём копию исходного поля для нормализации
df_deals["stage_normalized"] = df_deals["stage"]

# Применяем нормализацию
df_deals["stage_normalized"] = (
    df_deals["stage_normalized"]
    .replace(stage_mapping)
    .astype("string")
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)  # нормализация пробелов
    .str.title()
)

# Проверяем, какие значения не входят в стандартную воронку
invalid_stages = df_deals[
    ~df_deals["stage_normalized"].isin(valid_stages) &
    df_deals["stage_normalized"].notna()
]
invalid_count = len(invalid_stages)

if invalid_count == 0:
    print("Все значения 'stage_normalized' соответствуют стандартной воронке продаж.")
else:
    print(f"После расширения маппинга осталось нестандартных значений: {invalid_count}")
    print("Примеры:\n")
    print(invalid_stages["stage_normalized"].value_counts().head(10))

# Контроль итогового распределения стадий
print("\nРаспределение сделок по нормализованным стадиям:\n")
print(df_deals["stage_normalized"].value_counts(dropna=False))

print("\n=== 9. Коррекция перепутанных и некорректных сумм оплаты ===\n")

# Этап 9 направлен на исправление случаев, когда:
# - значение initial_amount_paid больше offer_total_amount (перепутаны местами);
# - либо сумма оплаты отрицательная (ошибка импорта).

# Перестановка перепутанных значений
mask_swap = (
    (df_deals["initial_amount_paid"] > df_deals["offer_total_amount"]) &
    (df_deals["offer_total_amount"].notna()) &
    (df_deals["initial_amount_paid"].notna())
)

swap_count = mask_swap.sum()
print(f"Обнаружено строк с возможной перестановкой значений: {swap_count}")

# Выполняем перестановку значений между initial_amount_paid и offer_total_amount
df_deals.loc[mask_swap, ["initial_amount_paid", "offer_total_amount"]] = (
    df_deals.loc[mask_swap, ["offer_total_amount", "initial_amount_paid"]].values
)

print(f"Перестановка значений выполнена в {swap_count} строках.")

# Замена отрицательных значений на пропуски
mask_negative = df_deals["initial_amount_paid"] < 0
neg_count = mask_negative.sum()

if neg_count > 0:
    df_deals.loc[mask_negative, "initial_amount_paid"] = pd.NA
    print(f"Отрицательные значения 'initial_amount_paid' заменены на NA ({neg_count} строк).")
else:
    print("Отрицательных значений 'initial_amount_paid' не обнаружено.")

# Контроль после коррекции
check_invalid = (df_deals["initial_amount_paid"] > df_deals["offer_total_amount"]).sum()

if check_invalid == 0:
    print("Все значения сумм оплаты логически корректны после коррекции.")
else:
    print(f"После коррекции осталось нарушений: {check_invalid} строк.")

# Проверка диапазона значений после исправлений
print("\nДиапазоны денежных значений после коррекции:\n")
for col in ["initial_amount_paid", "offer_total_amount"]:
    if col in df_deals.columns:
        min_val = df_deals[col].min()
        max_val = df_deals[col].max()
        missing = df_deals[col].isna().sum()
        print(f"{col:22} | Мин: {min_val:10} | Макс: {max_val:10} | Пропусков: {missing}")

print("\n=== 10. Проверка и нормализация поля 'quality' (расширенная версия для буквенных рейтингов) ===\n")

# Цель этапа:
# 1) Сохранить исходное поле quality для аудита;
# 2) Преобразовать CRM-рейтинг A–F в универсальные категории качества лидов;
# 3) Удалить возможные технические строки (если появятся).

# Список уникальных значений поля quality (для составления словаря маппинга)
unique_count = df_deals["quality"].nunique()
print(f"Количество уникальных значений поля quality: {unique_count}")
print(df_deals["quality"].unique())

# Проверим и удалим технические строки с quality='Test' (на будущее)
test_rows = df_deals[df_deals["quality"].astype(str).str.strip().str.lower() == "test"]
test_count = len(test_rows)
if test_count > 0:
    df_deals = df_deals[
        df_deals["quality"].astype(str).str.strip().str.lower() != "test"
    ].reset_index(drop=True)
    print(f"Удалено технических строк с quality='Test': {test_count}")
else:
    print("Технические строки с quality='Test' отсутствуют.")

# Словарь нормализации (адаптирован под CRM рейтинги A–E)
quality_mapping = {
    "A High": "Hot",
    "B Medium": "Warm",
    "C Low": "Cold",
    "D Non Target": "Cold",
    "E Non Qualified": "Undefined",
    "F": "Undefined",
    "Hot Lead": "Hot",
    "Warm Lead": "Warm",
    "Cold Lead": "Cold"
}

# Создаём новое поле для нормализованных значений
df_deals["quality_normalized"] = df_deals["quality"]

# Применяем нормализацию и очистку
df_deals["quality_normalized"] = (
    df_deals["quality_normalized"]
    .astype("string")
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
    .replace(quality_mapping)
)

# Проверяем, остались ли нестандартные категории
valid_quality = ["Hot", "Warm", "Cold", "Undefined"]
invalid_quality = df_deals[
    ~df_deals["quality_normalized"].isin(valid_quality) &
    df_deals["quality_normalized"].notna()
]
invalid_count = len(invalid_quality)

if invalid_count == 0:
    print("Все значения 'quality_normalized' корректно нормализованы.")
else:
    print(f"Осталось нестандартных значений: {invalid_count}")
    print("Примеры:")
    print(invalid_quality["quality_normalized"].value_counts().head(10))

# Контроль итогового распределения
print("\nРаспределение сделок по категориям 'quality_normalized':\n")
print(df_deals["quality_normalized"].value_counts(dropna=False))

print("\n=== 10.1 Обработка пропусков в 'quality_normalized' ===\n")

# Подсчёт количества пропусков
missing_quality = df_deals["quality_normalized"].isna().sum()
print(f"Количество пропусков в 'quality_normalized': {missing_quality}")

if missing_quality > 0:
    df_deals["quality_normalized"] = df_deals["quality_normalized"].fillna("Undefined")
    print(f"Пропуски заменены на 'Undefined'.")
else:
    print("Пропусков нет, дополнительных действий не требуется.")

# Контроль распределения после замены
print("\nРаспределение категорий 'quality_normalized' после обработки:\n")
print(df_deals["quality_normalized"].value_counts(dropna=False))

print("\n=== 11. Проверка логики обучения (course_duration и months_of_study) ===\n")

# Цель этапа:
# Проверить логическую согласованность между длительностью курса и количеством месяцев обучения.
# months_of_study не должен превышать course_duration и быть меньше 0.
# course_duration должен быть больше 0 и в разумных пределах (например, ≤ 60 месяцев).

# Проверка длительности курса
invalid_duration = df_deals[
    (df_deals["course_duration"].notna()) &
    ((df_deals["course_duration"] <= 0) | (df_deals["course_duration"] > 60))
]
print(f"Найдено записей с некорректной длительностью курса: {len(invalid_duration)}")

# Проверка количества месяцев обучения
invalid_study = df_deals[
    (df_deals["months_of_study"].notna()) &
    (
        (df_deals["months_of_study"] < 0) |
        (df_deals["months_of_study"] > df_deals["course_duration"])
    )
]
print(f"Найдено записей с некорректным количеством месяцев обучения: {len(invalid_study)}")

# Коррекция — замена аномалий на NA
if len(invalid_study) > 0:
    df_deals.loc[invalid_study.index, "months_of_study"] = pd.NA
    print(f"Значения 'months_of_study' в {len(invalid_study)} строках заменены на NA.")

if len(invalid_duration) > 0:
    df_deals.loc[invalid_duration.index, "course_duration"] = pd.NA
    print(f"Значения 'course_duration' в {len(invalid_duration)} строках заменены на NA.")

# Контроль после коррекции
print("\nОсновная статистика после проверки логики обучения:\n")
print(df_deals[["course_duration", "months_of_study"]].describe())

print("\n=== 12. Финальная проверка типов данных ===\n")

# Цель этапа:
# Убедиться, что все поля приведены к корректным типам данных
# перед сохранением очищенного набора.

# Проверяем текущие типы столбцов
print("Типы данных по столбцам:\n")
print(df_deals.dtypes)

# Проверяем наличие строковых полей, которые могли остаться object вместо string
object_cols = df_deals.select_dtypes(include="object").columns.tolist()
if object_cols:
    print("\nНайдены столбцы типа 'object':")
    print(object_cols)
    print("→ Преобразуем их к pandas.StringDtype()")
    df_deals[object_cols] = df_deals[object_cols].astype("string")
else:
    print("\nВсе строковые поля уже имеют корректный тип string.")

# Финальный контроль после преобразования
print("\nПроверка типов данных после преобразования:\n")
print(df_deals.dtypes)

print("\n=== 13. Выгрузка очищенного набора данных в .csv ===\n")

# Используем кастомную функцию export_dataframe_to_csv()
# Выгрузка очищенного набора данных
export_dataframe_to_csv(df_deals, "df_deals")


=== 1. Проверка и удаление полных дублей ===

[df_deals] Найдено полных дублей: 3
[df_deals] Полные дубли удалены. Размер после очистки: (21590, 23)

=== 2. Преобразование формата дат ===

[df_deals] Столбец 'created_time' преобразован в datetime64[ns]. Некорректных значений: 0
[df_deals] Столбец 'closing_date' преобразован в datetime64[ns]. Некорректных значений: 21590

=== 2.1 Коррекция столбца closing_date ===

После преобразования 'closing_date' пропусков: 21590

Распределение стадий среди сделок без даты закрытия:

stage
Lost                         15741
Call Delayed                  2248
Registered on Webinar         2072
Payment Done                   858
Waiting For Payment            325
Qualificated                   128
Registered on Offline Day      100
Need to Call - Sales            33
Need To Call                    31
Test Sent                       25
Need a consultation             23
New Lead                         5
Free Education                   1
Name: count, 

,id,deal_owner_name,closing_date,quality,stage,lost_reason,page,campaign,sla,content,...,product,education_type,created_time,course_duration,months_of_study,initial_amount_paid,offer_total_amount,contact_name,city,level_of_deutsch
1278,5.805028e+18,Ben Hall,NaT,C - Low,Call Delayed,NaN,/direct,blog2_DE,01:48:36,NaN,...,Web Developer,Morning,2024-06-06 14:53:00,6,<NA>,3000.0,2900.0,5.805028e+18,NaN,NaN
1392,5.805028e+18,Cara Iverson,NaT,D - Non Target,Lost,Non target,/eng,Berlin_DE,05:42:51,b6,...,UX/UI Design,Morning,2024-06-05 08:50:00,11,<NA>,11500.0,11000.0,5.805028e+18,Zwickau,NaN
1408,5.805028e+18,Charlie Davis,NaT,D - Non Target,Lost,Gutstein refusal,/eng,performancemax_eng_DE,13:22:11,_{region_name}_,...,UX/UI Design,Morning,2024-06-04 21:24:00,11,<NA>,11500.0,11000.0,5.805028e+18,Aschaffenburg,Б2
1439,5.805028e+18,Cara Iverson,NaT,A - High,Waiting For Payment,NaN,/direct,blog2_DE,00:39:03,NaN,...,UX/UI Design,Morning,2024-06-04 12:48:00,11,<NA>,11500.0,11000.0,5.805028e+18,Straubing,NaN
1451,5.805028e+18,Quincy Vincent,NaT,D - Non Target,Lost,Non target,/eng,Live_DE,00:18:50,b0,...,UX/UI Design,Morning,2024-06-04 11:13:00,11,<NA>,11500.0,11000.0,5.805028e+18,Augsburg,NaN
1483,5.805028e+18,Eva Kent,NaT,C - Low,Payment Done,NaN,/eng,24.09.23retargeting_DE,13:14:06,v15,...,Web Developer,Morning,2024-06-03 22:00:00,6,1,3000.0,2900.0,5.805028e+18,Rheine,b1
1702,5.805028e+18,Cara Iverson,NaT,C - Low,Waiting For Payment,NaN,/direct,blog2_DE,13:59:50,NaN,...,UX/UI Design,Morning,2024-05-31 21:07:00,11,<NA>,11500.0,11000.0,5.805028e+18,Perleberg,NaN
1966,5.805028e+18,Ulysses Adams,NaT,A - High,Lost,Changed Decision,/eng,performancemax_eng_DE,01:45:39,_{region_name}_,...,Web Developer,Morning,2024-05-28 12:34:00,6,<NA>,3000.0,2500.0,5.805028e+18,Kempten,NaN
2059,5.805028e+18,Cara Iverson,NaT,A - High,Waiting For Payment,NaN,/eng,22.05.2024wide_DE,02:44:09,bloggersvideo18com,...,UX/UI Design,Morning,2024-05-26 11:02:00,11,<NA>,11500.0,11000.0,5.805028e+18,Essen,NaN
2323,5.805028e+18,Cara Iverson,NaT,C - Low,Lost,NaN,/eng/test,24.09.23retargeting_DE,02:47:44,v15,...,UX/UI Design,Morning,2024-05-20 09:49:00,11,<NA>,11500.0,11000.0,5.805028e+18,Buxtehude,в1



=== 7. Очистка категориальных полей ===

Столбец 'stage' очищен и нормализован.
Столбец 'quality' очищен и нормализован.
Столбец 'payment_type' очищен и нормализован.
Столбец 'product' очищен и нормализован.
Столбец 'education_type' очищен и нормализован.
Столбец 'source' очищен и нормализован.
Столбец 'campaign' очищен и нормализован.
Столбец 'content' очищен и нормализован.
Столбец 'term' очищен и нормализован.
Столбец 'city' очищен и нормализован.
Столбец 'level_of_deutsch' очищен и нормализован.
Столбец 'page' очищен и нормализован.
Столбец 'lost_reason' очищен и нормализован.

Количество уникальных значений после очистки:
stage               : 13 уникальных значений
quality             : 6 уникальных значений
payment_type        : 3 уникальных значений
product             : 5 уникальных значений
education_type      : 2 уникальных значений

Проверка на наличие артефактных значений:

Типы данных после очистки категориальных полей:

stage               string[python]
quality        

'/content/drive/MyDrive/P. Project 07.11/csv/df_deals_clean_20251026_2136.csv'

## Предобработка df_spend (таблица Spend (Done).xlsx')



## Сопоставление владельцев контактов, звонков и сделок

In [317]:
print("=== Таблица сопоставления владельцев контактов, звонков и сделок ===\n")

# Получаем уникальные имена из всех трёх таблиц
contacts_owners = pd.Series(df_contacts["contact_owner_name"].unique(), name="contact_owner_name")
calls_owners = pd.Series(df_calls["call_owner_name"].unique(), name="call_owner_name")
deals_owners = pd.Series(df_deals["deal_owner_name"].unique(), name="deal_owner_name")

# Приведём к DataFrame и объединим для анализа
mapping_df = (
    pd.DataFrame(contacts_owners)
    .merge(pd.DataFrame(calls_owners), left_on="contact_owner_name", right_on="call_owner_name", how="outer")
    .merge(pd.DataFrame(deals_owners), left_on="contact_owner_name", right_on="deal_owner_name", how="outer", indicator=True)
)

# Сортировка для удобства анализа
mapping_df = mapping_df.sort_values(by="contact_owner_name", na_position="last").reset_index(drop=True)

# Итоги по количеству уникальных имён в каждом датасете
print(f"Всего уникальных имён в Contacts: {df_contacts['contact_owner_name'].nunique()}")
print(f"Всего уникальных имён в Calls: {df_calls['call_owner_name'].nunique()}")
print(f"Всего уникальных имён в Deals: {df_deals['deal_owner_name'].nunique()}")

# Поиск несовпадающих записей между тремя таблицами
non_matched = mapping_df[
    mapping_df["contact_owner_name"].isna()
    | mapping_df["call_owner_name"].isna()
    | mapping_df["deal_owner_name"].isna()
]

print(f"\nВсего несопоставленных записей: {len(non_matched)}")
print("\nПримеры несопоставленных записей:")
print(non_matched.head(15))

# Показываем итоговую таблицу сопоставления
print("\nТаблица сопоставления владельцев контактов, звонков и сделок (первые 40 строк):\n")
mapping_df.head(40)


=== Таблица сопоставления владельцев контактов, звонков и сделок ===

Всего уникальных имён в Contacts: 27
Всего уникальных имён в Calls: 33
Всего уникальных имён в Deals: 27

Всего несопоставленных записей: 10

Примеры несопоставленных записей:
   contact_owner_name call_owner_name deal_owner_name      _merge
6         Derek James     Derek James            <NA>   left_only
21         Tina Zhang      Tina Zhang            <NA>   left_only
27                NaN             NaN        John Doe  right_only
28                NaN             NaN     Xander Dean  right_only
29                NaN    Ethan Harris            <NA>        both
30                NaN   Fiona Jackson            <NA>        both
31                NaN      Hannah Lee            <NA>        both
32                NaN        John Doe            <NA>        both
33                NaN     Laura Quinn            <NA>        both
34                NaN     Xander Dean            <NA>        both

Таблица сопоставления владе

,contact_owner_name,call_owner_name,deal_owner_name,_merge
0,Alice Johnson,Alice Johnson,Alice Johnson,both
1,Amy Green,Amy Green,Amy Green,both
2,Ben Hall,Ben Hall,Ben Hall,both
3,Bob Brown,Bob Brown,Bob Brown,both
4,Cara Iverson,Cara Iverson,Cara Iverson,both
5,Charlie Davis,Charlie Davis,Charlie Davis,both
6,Derek James,Derek James,<NA>,left_only
7,Diana Evans,Diana Evans,Diana Evans,both
8,Eva Kent,Eva Kent,Eva Kent,both
9,George King,George King,George King,both
